#**Índicador da Média de alunos por turma em cada escola - 2020**
**Link dos Dados de origem:**
https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais/media-de-alunos-por-turma

Responsável: Gustavo de Andrade - gustavo.andrade@prefeitura.sp.gov.br

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime, date, timedelta

In [3]:
dados = pd.read_excel("ATU_ESCOLAS_2020.xlsx", skiprows = range(0, 8))
dados

,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,ED_INF_CAT_0,CRE_CAT_0,PRE_CAT_0,FUN_CAT_0,FUN_AI_CAT_0,FUN_AF_CAT_0,FUN_01_CAT_0,FUN_02_CAT_0,FUN_03_CAT_0,FUN_04_CAT_0,FUN_05_CAT_0,FUN_06_CAT_0,FUN_07_CAT_0,FUN_08_CAT_0,FUN_09_CAT_0,MULT_ETA_CAT_0,MED_CAT_0,MED_01_CAT_0,MED_02_CAT_0,MED_03_CAT_0,MED_04_CAT_0,MED_NS_CAT_0
0,2020,Norte,RO,1100015.0,Alta Floresta D'Oeste,11022558.0,EIEEF HAP BITT TUPARI,Rural,Estadual,--,--,--,1.5,1.5,--,--,1,1,2,2,--,--,--,--,--,--,--,--,--,--,--
1,2020,Norte,RO,1100015.0,Alta Floresta D'Oeste,11024275.0,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
2,2020,Norte,RO,1100015.0,Alta Floresta D'Oeste,11024291.0,EMMEF 7 DE SETEMBRO,Rural,Municipal,--,--,--,17,--,--,--,--,--,--,--,--,--,--,--,17,--,--,--,--,--,--
3,2020,Norte,RO,1100015.0,Alta Floresta D'Oeste,11024372.0,EMEIEF ANA NERY,Urbana,Municipal,8,--,8,13.1,15,11.5,--,--,14,15,16,9,13,11,13,14,--,--,--,--,--,--
4,2020,Norte,RO,1100015.0,Alta Floresta D'Oeste,11024666.0,EMEIEF BOA ESPERANCA,Rural,Municipal,12,--,12,20,19.3,21.3,--,--,14,22,22,16,28,21,20,17,--,--,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,2020,Centro-Oeste,DF,5300108.0,Brasília,53085000.0,COBIAN - COL BIANGULO,Urbana,Privada,17.7,12.8,22.5,23.3,23.3,--,26,19.3,26,--,--,--,--,--,--,--,--,--,--,--,--,--
175337,2020,Centro-Oeste,DF,5300108.0,Brasília,53086007.0,CED VITORIA REGIA,Urbana,Privada,--,--,--,21,--,21,--,--,--,--,--,--,--,17,25,--,29,29,25,33,--,--
175338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175339,Fonte: Censo da Educação Básica 2020/INEP.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filtrando - 'CO_MUNICIPIO' = 3550308 -> São Paulo

In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175341 entries, 0 to 175340
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   NU_ANO_CENSO    175340 non-null  object 
 1   NO_REGIAO       175338 non-null  object 
 2   SG_UF           175338 non-null  object 
 3   CO_MUNICIPIO    175338 non-null  float64
 4   NO_MUNICIPIO    175338 non-null  object 
 5   CO_ENTIDADE     175338 non-null  float64
 6   NO_ENTIDADE     175338 non-null  object 
 7   NO_CATEGORIA    175338 non-null  object 
 8   NO_DEPENDENCIA  175338 non-null  object 
 9   ED_INF_CAT_0    175338 non-null  object 
 10  CRE_CAT_0       175338 non-null  object 
 11  PRE_CAT_0       175338 non-null  object 
 12  FUN_CAT_0       175338 non-null  object 
 13  FUN_AI_CAT_0    175338 non-null  object 
 14  FUN_AF_CAT_0    175338 non-null  object 
 15  FUN_01_CAT_0    175338 non-null  object 
 16  FUN_02_CAT_0    175338 non-null  object 
 17  FUN_03_CAT

In [9]:
indicador_media_por_turma = dados.loc[lambda dados: dados["NO_MUNICIPIO"] == "São Paulo"]
indicador_media_por_turma

,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,ED_INF_CAT_0,CRE_CAT_0,PRE_CAT_0,FUN_CAT_0,FUN_AI_CAT_0,FUN_AF_CAT_0,FUN_01_CAT_0,FUN_02_CAT_0,FUN_03_CAT_0,FUN_04_CAT_0,FUN_05_CAT_0,FUN_06_CAT_0,FUN_07_CAT_0,FUN_08_CAT_0,FUN_09_CAT_0,MULT_ETA_CAT_0,MED_CAT_0,MED_01_CAT_0,MED_02_CAT_0,MED_03_CAT_0,MED_04_CAT_0,MED_NS_CAT_0
130920,2020,Sudeste,SP,3550308.0,São Paulo,35000012.0,AYRES DE MOURA PROFESSOR,Urbana,Estadual,--,--,--,34.8,--,34.8,--,--,--,--,--,35,35,35.8,33.5,--,--,--,--,--,--,--
130921,2020,Sudeste,SP,3550308.0,São Paulo,35000024.0,GAVIAO PEIXOTO BRIGADEIRO,Urbana,Estadual,--,--,--,27.3,26.8,27.7,26.5,29.7,26.2,24.3,28.3,20,31.5,32,35,--,33.5,34.3,39,26.5,--,--
130922,2020,Sudeste,SP,3550308.0,São Paulo,35000036.0,JOAO SOLIMEO,Urbana,Estadual,--,--,--,33.4,--,33.4,--,--,--,--,--,36.4,33,30.1,35.4,--,34.4,35.2,35,32,--,--
130923,2020,Sudeste,SP,3550308.0,São Paulo,35000048.0,WALFREDO ARANTES CALDAS PROFESSOR,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,34.9,34.1,36.3,34.2,--,--
130924,2020,Sudeste,SP,3550308.0,São Paulo,35000059.0,SEBASTIAO DE OLIVEIRA GUSMAO PROFESSOR,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,30.1,25.8,37.4,30,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137910,2020,Sudeste,SP,3550308.0,São Paulo,35925998.0,ADELAIDE ROSA FERNANDES MACHADO DE SOUZA PROFE...,Urbana,Estadual,--,--,--,31.3,--,31.3,--,--,--,--,--,37,35,32,21,--,28.7,27,29,36,--,--
137911,2020,Sudeste,SP,3550308.0,São Paulo,35926048.0,BARRO BRANCO II,Urbana,Estadual,--,--,--,32.6,31.7,33.9,32.5,32.8,33.5,31.2,28.3,35.6,28.8,34.3,37.7,--,31.5,25.9,36.5,41.5,--,--
137912,2020,Sudeste,SP,3550308.0,São Paulo,35926103.0,PROFESSORA ELIANA PASSUELO,Urbana,Estadual,--,--,--,29.2,29.2,--,29.3,29.7,29.7,28.4,29.5,--,--,--,--,--,--,--,--,--,--,--
137913,2020,Sudeste,SP,3550308.0,São Paulo,35980018.0,CEEJA CLARA MANTELLI DONA,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


**Verificadores de conteúdo**

In [10]:
len(indicador_media_por_turma)

6995

In [11]:
indicador_media_por_turma.NU_ANO_CENSO.unique()

array([2020], dtype=object)

In [14]:
indicador_media_por_turma.NO_DEPENDENCIA.unique() 

array(['Estadual', 'Municipal', 'Privada', 'Federal'], dtype=object)

**Limpando colunas**

In [12]:
indicador_media_por_turma.drop(columns =['NU_ANO_CENSO','NO_REGIAO','SG_UF','CO_MUNICIPIO','NO_MUNICIPIO'], inplace = True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [17]:
indicador_media_por_turma = indicador_media_por_turma.rename(columns = {'CO_ENTIDADE':'id_escola', 'NO_ENTIDADE':'escola','NO_CATEGORIA':'categoria', 'NO_DEPENDENCIA':'tipo' }, inplace = False)

In [20]:
indicador_media_por_turma = indicador_media_por_turma.rename(columns = {'ED_INF_CAT_0':'total_ens_infantil', 'CRE_CAT_0':'creche_ens_infantil', 'PRE_CAT_0'	: 'pre_escola_ens_infantil' }, inplace = False)

In [22]:
indicador_media_por_turma = indicador_media_por_turma.rename(columns = {'FUN_CAT_0':'total_ens_fund', 'FUN_AI_CAT_0':'total_anos_inciais_ens_fund', 'FUN_AF_CAT_0': 'total_anos_finais_ens_fund','FUN_01_CAT_0':'ano1_ens_fund','FUN_02_CAT_0':'ano2_ens_fund', 'FUN_03_CAT_0':'ano3_ens_fund', 'FUN_04_CAT_0':'ano4_ens_fund', 'FUN_05_CAT_0':'ano5_ens_fund', 'FUN_06_CAT_0':'ano6_ens_fund', 'FUN_07_CAT_0':'ano7_ens_fund', 'FUN_08_CAT_0':'ano8_ens_fund', 'FUN_09_CAT_0':'ano9_ens_fund'}, inplace = False)

In [24]:
indicador_media_por_turma = indicador_media_por_turma.rename(columns = {'MULT_ETA_CAT_0':'turma_multi_estapas_ens_fund'}, inplace = False)

In [26]:
indicador_media_por_turma = indicador_media_por_turma.rename(columns = {'MED_CAT_0':'total_ens_medio','MED_01_CAT_0':'ano1_ens_med','MED_02_CAT_0':'ano2_ens_med','MED_03_CAT_0':'ano3_ens_med','MED_04_CAT_0':'ano4_ens_med','MED_NS_CAT_0':'nao_seriado_ens_medio' }, inplace = False)

#**Tabela Dimensional: media_de_alunos_escolas_dim**

In [28]:
media_de_alunos_escolas_dim = pd.DataFrame()
media_de_alunos_escolas_dim['escola'] = indicador_media_por_turma['escola']
media_de_alunos_escolas_dim['id_escola'] = indicador_media_por_turma['id_escola']

In [29]:
media_de_alunos_escolas_dim

,escola,id_escola
130920,AYRES DE MOURA PROFESSOR,35000012.0
130921,GAVIAO PEIXOTO BRIGADEIRO,35000024.0
130922,JOAO SOLIMEO,35000036.0
130923,WALFREDO ARANTES CALDAS PROFESSOR,35000048.0
130924,SEBASTIAO DE OLIVEIRA GUSMAO PROFESSOR,35000059.0
...,...,...
137910,ADELAIDE ROSA FERNANDES MACHADO DE SOUZA PROFE...,35925998.0
137911,BARRO BRANCO II,35926048.0
137912,PROFESSORA ELIANA PASSUELO,35926103.0
137913,CEEJA CLARA MANTELLI DONA,35980018.0


In [31]:
media_de_alunos_escolas_dim.to_csv(r'media_de_alunos_escolas_dim', index = False)

#**Concluindo a Fato: media_de_alunos_escolas_fato**

In [32]:
indicador_media_por_turma.drop(columns =['escola'], inplace = True) 

In [33]:
indicador_media_por_turma

,id_escola,categoria,tipo,total_ens_infantil,creche_ens_infantil,pre_escola_ens_infantil,total_ens_fund,total_anos_inciais_ens_fund,total_anos_finais_ens_fund,ano1_ens_fund,ano2_ens_fund,ano3_ens_fund,ano4_ens_fund,ano5_ens_fund,ano6_ens_fund,ano7_ens_fund,ano8_ens_fund,ano9_ens_fund,turma_multi_estapas_ens_fund,total_ens_medio,ano1_ens_med,ano2_ens_med,ano3_ens_med,ano4_ens_med,nao_seriado_ens_medio
130920,35000012.0,Urbana,Estadual,--,--,--,34.8,--,34.8,--,--,--,--,--,35,35,35.8,33.5,--,--,--,--,--,--,--
130921,35000024.0,Urbana,Estadual,--,--,--,27.3,26.8,27.7,26.5,29.7,26.2,24.3,28.3,20,31.5,32,35,--,33.5,34.3,39,26.5,--,--
130922,35000036.0,Urbana,Estadual,--,--,--,33.4,--,33.4,--,--,--,--,--,36.4,33,30.1,35.4,--,34.4,35.2,35,32,--,--
130923,35000048.0,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,34.9,34.1,36.3,34.2,--,--
130924,35000059.0,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,30.1,25.8,37.4,30,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137910,35925998.0,Urbana,Estadual,--,--,--,31.3,--,31.3,--,--,--,--,--,37,35,32,21,--,28.7,27,29,36,--,--
137911,35926048.0,Urbana,Estadual,--,--,--,32.6,31.7,33.9,32.5,32.8,33.5,31.2,28.3,35.6,28.8,34.3,37.7,--,31.5,25.9,36.5,41.5,--,--
137912,35926103.0,Urbana,Estadual,--,--,--,29.2,29.2,--,29.3,29.7,29.7,28.4,29.5,--,--,--,--,--,--,--,--,--,--,--
137913,35980018.0,Urbana,Estadual,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


In [34]:
indicador_media_por_turma.to_csv(r'media_de_alunos_escolas_fato', index = False)